In [1]:
import numpy as np
import ray.rllib.agents.ppo as ppo
import ray.rllib.agents.dqn as dqn
import ray.rllib.agents.a3c as a3c
from ray.tune.logger import pretty_print
import gym
from gym.utils.env_checker import check_env

In [2]:
import ray
ray.shutdown()
ray.init()

RayContext(dashboard_url='', python_version='3.9.11', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2023-09-05_18-28-34_482700_52968/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2023-09-05_18-28-34_482700_52968/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2023-09-05_18-28-34_482700_52968', 'metrics_export_port': 62490, 'gcs_address': '127.0.0.1:63008', 'address': '127.0.0.1:63008', 'node_id': 'f9175ab2d6eaec976fbee1c156b6a76835fc6e08c77b81805ec30c95'})

In [69]:
config = dqn.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["framework"] = "torch"
config["env_config"] = {}
# config['kl_coeff'] = 0.0

In [70]:
B = np.array([786,759,888,649]).astype(np.float64)
A = np.array([[300,400,10,50,900],
              [200,400,10,50,900],
              [100,400,10,50,900],
              [ 50,400,10,50,900]]).astype(np.float64)

In [71]:
class MyEnv(gym.Env):
    def __init__(self, env_config):
        super().__init__()
        self.action_space = gym.spaces.Discrete(5)
        self.observation_space = gym.spaces.Box(low=-1000, high=1000, shape=(4,5), dtype=np.float64)
        self.aa = np.array(A)
        self.bb = np.array(B)
        self.state = self.bb[:,np.newaxis] - self.aa
        self.done = False

    def reset(self):
        self.aa = np.array(A)
        self.bb = np.array(B)
        self.state = self.bb[:,np.newaxis] - self.aa
        self.done = False
        return self.state

    def step(self, action):
        if np.any(self.state < 0, axis = 0)[action]:
            self.reward = -50.0
            self.done = False
        else:
            self.bb = self.bb - self.aa[:,action]
            self.aa[:,action] = self.bb + 1
            self.reward = np.sum(A[:,action])/100.0
            self.state = self.bb[:,np.newaxis] - self.aa
            if np.all(self.state < 0):
                self.done = True
            else:
                self.done = False
        return self.state, self.reward, self.done, {}

In [72]:
# agent3 = ppo.PPOTrainer(config=config, env=MyEnv)
agent3 = dqn.DQNTrainer(config=config, env=MyEnv)
# agent3 = a3c.A3CTrainer(config=config, env=MyEnv)

(RolloutWorker pid=57603) 2023-09-05 19:43:44,503	WARNING rollout_worker.py:498 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).
(RolloutWorker pid=57603) 2023-09-05 19:43:44,503	WARNING env.py:120 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2023-09-05 19:43:44,535	WARNING util.py:60 -- Install gputil for GPU system monitoring.


In [73]:
for i in range(10):
   # Perform one iteration of training the policy with PPO
   result = agent3.train()
   if i % 2 == 0 or i == 49:
       # print(pretty_print(result))
       print('i: ', i)
       print('mean episode length:', result['episode_len_mean'])
       print('max episode reward:', result['episode_reward_max'])
       print('mean episode reward:', result['episode_reward_mean'])
       print('min episode reward:', result['episode_reward_min'])
       print('total episodes:', result['episodes_total'])
       print()
       # checkpoint = agent3.save()
       # print("checkpoint saved at", checkpoint)

i:  0
mean episode length: 10.505263157894737
max episode reward: 24.9
mean episode reward: -300.3631578947367
min episode reward: -1125.1
total episodes: 95

i:  2
mean episode length: 11.97
max episode reward: 24.9
mean episode reward: -373.6
min episode reward: -1825.1
total episodes: 266

i:  4
mean episode length: 12.42
max episode reward: 24.9
mean episode reward: -396.0999999999999
min episode reward: -1825.1
total episodes: 431

i:  6
mean episode length: 16.07
max episode reward: 24.9
mean episode reward: -578.6
min episode reward: -3875.1
total episodes: 553

i:  8
mean episode length: 26.81
max episode reward: 24.9
mean episode reward: -1115.6000000000004
min episode reward: -11225.1
total episodes: 616



In [74]:
env = MyEnv(config)
state = env.reset()
g = 0
done = False
reward = 0
i = 0
while (not done) and i < 50:
  action = agent3.compute_action(state, explore = False)
  state, reward, done, info = env.step(action)
  print(f"state = {state} action = {action} reward = {reward} done = {done}")
  g += reward
  i += 1
print(g)

state = [[ 436.  336.  726.   -1. -164.]
 [ 509.  309.  699.   -1. -191.]
 [ 738.  438.  828.   -1.  -62.]
 [ 549.  199.  589.   -1. -301.]] action = 3 reward = 2.0 done = False
state = [[  36.   -1.  326. -401. -564.]
 [ 109.   -1.  299. -401. -591.]
 [ 338.   -1.  428. -401. -462.]
 [ 149.   -1.  189. -401. -701.]] action = 1 reward = 16.0 done = False
state = [[  -1. -301.   26. -701. -864.]
 [  -1. -201.   99. -601. -791.]
 [  -1. -101.  328. -501. -562.]
 [  -1.  -51.  139. -451. -751.]] action = 0 reward = 6.5 done = False
state = [[ -11. -311.   -1. -711. -874.]
 [ -11. -211.   -1. -611. -801.]
 [ -11. -111.   -1. -511. -572.]
 [ -11.  -61.   -1. -461. -761.]] action = 2 reward = 0.4 done = True
24.9
